In [26]:
import pandas as pd
import numpy as np
import os
import time
import logging
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import openpyxl
import json
import io
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


# pd.set_option('display.max_colwidth', None)
#logger
logging.basicConfig(filename='WB governance.log', filemode='w', format='%(asctime)s - %(message)s', level=logging.INFO)

#### Getting country list

In [2]:
#for WGI
df_countries=pd.read_excel('Countries.xlsx')
df_countries=df_countries[(df_countries['Filter']==1)&(df_countries['dataset']=='Governance_dataset')]
countries_wgi=dict(zip(df_countries['iso3'],df_countries['Country']))

#for ISPAR
df_countries=pd.read_excel('Countries.xlsx')
df_countries_ispar=df_countries[(df_countries['Filter']==1)&(df_countries['dataset']=='ISPAR')]

#for TC360
df_countries=pd.read_excel('Countries.xlsx')
df_countries=df_countries[(df_countries['Filter']==1)&(df_countries['dataset']=='TCdata360')]
countries_tc360=dict(zip(df_countries['iso3'],df_countries['Country']))

#### WebScraping from Global Knowledge Index

In [3]:
logging.info('#############downloading Global Knowledge Index from https://www.knowledge4all.com/ranking ##############')

In [31]:
#get the API call from inspect page--network--fetch/XHR--click on the table--right click on the visible call and copy as fetch
#transform the java script to python in chatgpt

url = "https://www.knowledge4all.com/_next/data/YOrK06OWNaVAl_cfMrF5z/en/ranking.json?Id=1&Color=00248b"

headers = {
    "accept": "*/*",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8,ar;q=0.7,fr;q=0.6,ru;q=0.5",
    "sec-ch-ua": "\"Chromium\";v=\"116\", \"Not)A;Brand\";v=\"24\", \"Google Chrome\";v=\"116\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "Referer": "https://www.knowledge4all.com/ranking",
    "referrerPolicy": "strict-origin-when-cross-origin"
}

response = requests.get(url, headers=headers, cookies={"credentials": "include"})

# Check the response
if response.status_code == 200:
    json_data = response.json()
    print(json_data)
else:
    print("Request failed with status code:", response.status_code)


{'pageProps': {'_nextI18Next': {'initialI18nStore': {'en': {'knowledge-for-all': {}, 'common': {}}}, 'userConfig': {'i18n': {'defaultLocale': 'en', 'locales': ['en', 'ar'], 'localePath': 'C:\\inetpub\\wwwroot\\K4All\\Web\\public\\locales'}, 'react': {'useSuspense': False, 'wait': True}, 'default': {'i18n': {'defaultLocale': 'en', 'locales': ['en', 'ar'], 'localePath': 'C:\\inetpub\\wwwroot\\K4All\\Web\\public\\locales'}, 'react': {'useSuspense': False, 'wait': True}}}}, 'initialState': {'countryReducer': {'countries': [], 'country': {'countryCode': '', 'countryName': '', 'countryId': '', 'flag': '', 'file': ''}, 'countryInfo': {}, 'tableValues': [], 'gKIsValues': [], 'radarChartValues': [], 'mainChartValues': []}, 'rankingReducer': {'countries': [{'id': 1002, 'name': 'Albania', 'isoCode2': 'AL', 'isoCode3': 'ALB', 'flagURL': 'https://knowledge4all.com/admin/CountryFlags/ALB.jpg'}, {'id': 1122, 'name': 'Argentina', 'isoCode2': None, 'isoCode3': 'ARG', 'flagURL': 'https://knowledge4all.c

In [27]:
url='https://www.knowledge4all.com/ranking'

driver = webdriver.Chrome('C:/Users/511232/Desktop/Worldwide Governance Indicators/chromedriver-win64/chromedriver.exe')
driver.maximize_window()

driver.get(url)
# driver.implicitly_wait(15)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="__next"]/div/div[3]/div/div/div/div/section/table/tbody')))

<ipython-input-27-764b4c42fdca>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/511232/Desktop/Worldwide Governance Indicators/chromedriver-win64/chromedriver.exe')


<selenium.webdriver.remote.webelement.WebElement (session="b4c7ddb05e2fe4676f88656899b797ac", element="8DD1C3B21A5380C64A475C3DDDD3C537_element_25")>

In [28]:
driver.find_elements_by_xpath('//*[@id="__next"]/div/div[3]/div/div/div/div/section/table/tbody')
# driver.find_element_by_xpath('/html/body/div/div[2]/div[4]/div[2]/button').click()

<ipython-input-28-c792c41bc034>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_xpath('//*[@id="__next"]/div/div[3]/div/div/div/div/section/table/tbody')


[<selenium.webdriver.remote.webelement.WebElement (session="b4c7ddb05e2fe4676f88656899b797ac", element="8DD1C3B21A5380C64A475C3DDDD3C537_element_25")>]

#### Bulk download from WGI

In [43]:
logging.info('#############downloading from https://info.worldbank.org/governance/wgi/#home ##############')

url='https://info.worldbank.org/governance/wgi/#home'

#get the url for download
response=requests.get(url)
logging.info(f'request status code for the main url {response.status_code}')
soup=BeautifulSoup(response.content, 'html.parser')

#getting the href for Download full dataset (Excel) link in the webpage
s1=soup.find_all("ul", {"class": "listItems"})

url=s1[1].find_all("a", {"class": "file-download"})[2]['href']
domain='https://info.worldbank.org'
#construct the url
bulk_download_url=domain+url
#to download the url
try:
    r=requests.get(bulk_download_url)
    logging.info(f'Download request is status code is {r.status_code}')
except Exception as e:
    logging.warning(e)
    raise e

#write the raw data to excel
with open('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/WB governance.xlsx','wb') as f:
    try:
        f.write(r.content)
        logging.info('successfuly downloaded bulk excel file')
    except Exception as e:
        logging.warning(e)
logging.info('---------------------------------------------------')

try:
    df_WGI=pd.DataFrame()

    #get the sheet names to loop over
    wb=openpyxl.load_workbook("C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/WB governance.xlsx")
    sheetnames=wb.sheetnames[1:]

    #counter to count rows of newly added dataframes
    sheet_counter={}

    for sh in sheetnames:
        logging.info(f'processing sheet {sh}')    
        df=pd.read_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/WB governance.xlsx', sheet_name=sh)

        #EXTRACT THE DATAFRAME FROM UNSTRCUTURED EXCEL SHEET
        #get the dataframe filtered on idx at value = "Country/Territory"
        idx=df[df.iloc[:,0]=="Country/Territory"].index[0]
        df1=df.iloc[idx-1:, :].reset_index(drop=True)

        #zip the 1st 2 rows to make columns. output example as below
        '''((nan, 'Country/Territory'),
        (nan, 'Code'),
        (1996, 'Estimate'),
        (1996, 'StdErr'),
        (1996, 'NumSrc'),'''
        cols=tuple(zip(df1.iloc[0,:],df1.iloc[1,:]))
        #drop the 2 rows
        df1.drop(axis=0, index=[0,1], inplace=True)
        #add the cols as columns
        df1.columns=cols
        
        #add the sheet name as a column
        df1[sh]=sh

        #filter the dataframe on ['Country/Territory','Code','Estimate', 'Rank']
        cols_to_filter=[(a,b) for (a,b) in list(df1.columns) if (b in ['Country/Territory','Code','Estimate', 'Rank']) | (a==sh)]
        #cols_to_filter will look like
        '''[(nan, 'Country/Territory'),
        (nan, 'Code'),
        (1996, 'Estimate'),
        (1996, 'Rank'),'''
        df1=df1.loc[:,cols_to_filter]

        #'Estimate', 'Rank' will be melted, so the melted dataframe must have double the row numbers of the unmelted dataframe. the number of  'Estimate', 'Rank' for timeseries = len(cols_to_filter) - 3 (remove 'Country/Territory','Code', sh)
        wide_df_rows= len(df1)
        n_wide=len(cols_to_filter)-3

        #RESHAPE THE DATAFRAME
        #choose 'Country/Territory','Code' and the sheetname col added at the end of the dataframe as id_vars (vars not to be reshaped) and the rest as value_vars (variables to reshape)
        # id_variables such as: [(nan, 'Country/Territory'), (nan, 'Code'), (2021, 'Rank')]
        id_variables=df1.columns.to_list()[:2]
        id_variables.append(df1.columns.to_list()[-1])

        df_melt=pd.melt(df1,
                id_vars=id_variables,
                value_vars=df1.columns.to_list()[2:-1],
                var_name=['Year','Value'])
                
        df_melt.columns=['Country','Code','Indicator','Year','Type','Value']
        
        #check if melting went ok by checking the number of rows pre and post melting
        if len(df_melt)==wide_df_rows*n_wide:
            logging.info(f'sheet {sh} succesfully reshaped to long with {len(df_melt)} rows')
        else:
            logging.warning(f'WARNING!!!!!!!!! problem with melting {sh}, which has {len(df_melt)} rows whereas before melting it had {wide_df_rows} rows and n_wide was {n_wide}')

        #APPEND TO Governance_total DATAFRAME
        df_WGI=df_WGI.append(df_melt)
        logging.info(f'sheet {sh} is appended successfully')
        logging.info(f'Governance_total has {len(df_WGI)} rows')
        logging.info('---------------------------------------------------------')

    #adding the world average
    df_WGI['Value']=pd.to_numeric(df_WGI['Value'])
    world_avg=df_WGI.groupby(['Indicator', 'Year']).agg({'Value':'mean'}).reset_index()
    world_avg['Country']='World'
    world_avg
    Governance_dataset=pd.concat([df_WGI,world_avg], axis=0)
    logging.info('World averages added to the Governance_dataset')

    #transforming indicator names
    ind_dct={'VoiceandAccountability':'Voice and Accountability',
             'Political StabilityNoViolence' : 'Political Stability No Violence',
             'GovernmentEffectivenes' : 'Government Effectiveness',
             'RegulatoryQuality' : 'Regulatory Quality',
             'RuleofLaw' : 'Rule of Law',
             'ControlofCorruption' : 'Control of Corruption'}
    
    df_WGI['Indicator']=df_WGI['Indicator'].map(ind_dct)

    df_WGI.to_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/df_WGI.xlsx', index=False)
    logging.info(f'Governance_dataset saved with {len(df_WGI)} rows')

    #filter on Arab countries and world
    df_WGI_Arab=df_WGI[df_WGI['Country'].isin(countries_wgi.values())].copy()
    
    #rename Code to ISO
    df_WGI_Arab.rename({'Code':'ISO'}, axis=1, inplace=True)
    #change WBG to PSE for State of Palestine
    df_WGI_Arab.loc[df_WGI_Arab['ISO']=='WBG', 'ISO']='PSE'
    logging.info('Code changed from WBG to PSE for State of Palestine')
    df_WGI_Arab.loc[df_WGI_Arab['ISO']=='PSE', 'Country']='State of Palestine'
    logging.info('Country/Territory changed from WBG to PSE for State of Palestine')

    #add Note regarding State of Palestine
    df_WGI_Arab.loc[df_WGI_Arab['ISO']=='PSE', 'Note']='Only for West Bank and Gaza'
    logging.info('West bank and Gaza note is added')
    
    #calculate Arab World average and add to the dataframe
    def get_avg_for_Arab(df):
                    result=(df['Value'].mean(),'ARB','Arab World','ESCWA estimate','WGI')
                    return(pd.Series(result))

    df_WGI_Arab_AVG=df_WGI_Arab.groupby(['Indicator','Type']).apply(get_avg_for_Arab).reset_index()
    df_WGI_Arab_AVG.columns=['Indicator','Type','Value','ISO','Country','Note','Source']
    logging.info(f'Arab world values calculated by grouping by "[Indicator,Type]" ')
    
    #Add Source
    df_WGI_Arab['Source']='WGI'
    #concatenate with original file
    df_WGI_Arab=pd.concat([df_WGI_Arab,df_WGI_Arab_AVG], axis=0)
    logging.info('Arab World averages concatenated with the original file')

    df_WGI_Arab.to_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/df_WGI_Arab.xlsx', index=False)
    logging.info(f'Governance_dataset_Arab saved with {len(df_WGI_Arab)} rows')
    logging.info('///////////////////////////////////////////////////////\n')
except Exception as e:
    logging.warning(f'!!! WARNING !!! {e}')
    raise e

#### ISPAR

In [37]:
logging.info('#############getting data from ISPAR##############')

id={3:'Global Cybersecurity Index',
        4:'Global Gender Gap Index',
        5:'ICT Development Index',
        6:'Network Readiness Index',
        7:'Global Innovation Index',
        9:'E-Government Development Index',
        11:'Global Competitiveness Index',
        12:'E-Participation index',
        14:'AI Readiness Index',
        16:'Open Data Inventory Index'}

url='https://datacatalog.unescwa.org/datastore/dump_v2?resource_id=e0d88222-f90c-4a92-8de4-397f6529c402&format=csv'

#get the url for download
response=requests.get(url)

try:
    if response.status_code!=200:
        print(response.status_code)
        logging.warning(f'!!!!!!WARNING!!!!!!status code is {response.status_code}')

    #read the response into pandas dataframe
    df = pd.read_csv(io.BytesIO(response.content))
    print(f'dataframe read in {len(df)} rows')
    logging.info(f'dataframe returned with {len(df)} rows')

    #remove the 0.colname from column names
    df.columns=[i.replace("0.", "") for i in df.columns]
    #rename Index Name to Indicator, Country Name to Country
    df.rename({'Index Name':'Indicator', 'Country Name':'Country'}, axis=1, inplace=True)
    df['Source']='ISPAR'
    #save ISPAR_all countries
    df.to_excel('ISPAR_all countries.xlsx', index=False)

    #filter on Index ID and 'Country Name'
    df_ispar_arab=df[(df['Index ID'].isin(id)) & (df['Country'].isin(df_countries_ispar['Country']))]

    #change PSE to State of Palestine
    df_ispar_arab.loc[df_ispar_arab['ISO']=='PSE', 'Country']='State of Palestine'
    logging.info('Country changed from West Bank and Gaza to State of Palestine')

    #add Note regarding State of Palestine
    df_ispar_arab.loc[df_ispar_arab['ISO']=='PSE', 'Note']='Only for West Bank and Gaza'
    logging.info('West bank and Gaza note is added')

    df_ispar_arab.to_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/ISPAR_Arab.xlsx', index=False)
    logging.info(f'dataframe for Arab countries returned with {len(df_ispar_arab)} rows')

    #goupby and aggregate as Score mean
    def group_agg(df):
        return (df['Score'].mean(), list(df['Country'].unique()))

    df_ispar_arab_agg=df_ispar_arab.groupby(['Indicator', 'Year']).apply(group_agg).to_frame().reset_index()
    df_ispar_arab_agg.columns=['Indicator','Year', 'result']

    df_ispar_arab_agg['Value']=df_ispar_arab_agg['result'].map(lambda x: x[0])
    df_ispar_arab_agg['Country group']=df_ispar_arab_agg['result'].map(lambda x: x[1])
    df_ispar_arab_agg.drop(['result'], axis=1, inplace=True)
    df_ispar_arab_agg.head()

    df_ispar_arab_agg['Country']='Arab World'
    df_ispar_arab_agg.to_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/ISPAR_AGGREGATED.xlsx', index=False)
    logging.info('groupby.aggregate() successful and saved as ISPAR_AGGREGATED.xlsx')

    # #to download to a csv
    # with open('ISPAR.csv', 'wb') as f:
    #     f.write(response.content)

except Exception as e:
    logging.warning(f'!!!WARNING!!!  {e}')
    raise e

logging.info('///////////////////////////////////////////////////////\n')

dataframe read in 43990 rows


c:\Users\511232\Desktop\Worldwide Governance Indicators\.WGI\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
c:\Users\511232\Desktop\Worldwide Governance Indicators\.WGI\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


### Worldbank  TCdata360

In [5]:
try:

    logging.info('#############creating the url for API request for TCdata360##############')

    ind_url='https://tcdata360-backend.worldbank.org/api/v1/indicators/'
    #get the url for download
    response=requests.get(ind_url)
    logging.info(f'status code for response ~/api/v1/indicators/ is {response.status_code}')

    #get indicator codes to add to the url
    df_ind=pd.read_excel('TCdata360_all indicators.xlsx')
    df_ind=df_ind[df_ind['Filter']==1]
    ind=dict(zip(df_ind['id'],df_ind['name']))

    main_data_url='https://tcdata360-backend.worldbank.org/api/v1/data?'

    # m_data_url='https://tcdata360-backend.worldbank.org/api/v1/data?countries=ARE%2CBHR%2CCOM%2CDJI%2CDZA%2CEGY%2CIRQ%2CJOR%2CKWT%2CLBN%2CLBY%2CMAR%2CMRT%2COMN%2CQAT%2CSAU%2CSDN%2CSOM%2CSYR%2CTUN%2CYEM%2CPSE&indicators=45410%2C45283%2C3294%2C1776%2C799%2C3500%2C960%2C3421%2C922%2C2073%2C27959%2C27962%2C40712%2C40711'


    cntry_url=''
    first_element=True
    for k,v in countries_tc360.items():
        if first_element:
            cntry_url=cntry_url+k
            first_element=False
        else:
            cntry_url=cntry_url+'%2C'+k


    ind_url=''
    first_element=True
    for k,v in ind.items():
        if first_element:
            ind_url=ind_url+str(k)
            first_element=False
        else:
            ind_url=ind_url+'%2C'+str(k)

    data_url=main_data_url+'countries='+cntry_url+'&'+'indicators='+ind_url
    logging.info(f'data_url constructed as {data_url}')

except Exception as e:
    logging.warning(f' !!! WARNING !!! {e}')
    raise

############################################################
################get time series data #######################
try:
    logging.info('#############getting timeseries data for TCdata360##############')
    response=requests.get(data_url)
    logging.info(f'status code for response ~/api/v1/indicators/ is {response.status_code}')
    r=json.loads(response.content)

    '''r['data'] is a list of nested dictionaries of the below structure
    [{'id': 'DZA',
    'indicators': [{'id': 799,
        'values': {'1988': 6.3, '1995': 5.6, '2011': 0.4},
        'estimated': []}, ...'''

    final_data=[]
    #looping over each country data
    for c in r['data']:
        country=c['id']
        #looping over data to get metadata
        for m in c['indicators']:
            metadata=[m['id'],m['estimated']]
            metadata.append(country)
            #looping over data to get time series values and append to eat list above metadata
            country_data=[]
            for v in m['values'].items():
                datapoint=list(v)
                datapoint.extend(metadata)
                country_data.append(datapoint)

            final_data.extend(country_data)
                
    df_tc360=pd.DataFrame(final_data,columns=['Year','Value','indicator_id','estimated','Country'])
    logging.info(f'dataframe for time series created successfuly with  {len(df_tc360)} rows')

    # create a column for indicator name
    df_tc360['Indicator_name']=df_tc360['indicator_id'].map(ind)
    logging.info('Indicator_name created')
    df_tc360['country_name']=df_tc360['Country'].map(countries_tc360)
    logging.info('country_name created')

    #rename columns
    df_tc360.rename({'Country': 'ISO', 'country_name':'Country', 'Indicator_name':'Indicator'}, axis=1, inplace=True)
    
    #change PSE to State of Palestine
    df_tc360.loc[df_tc360['ISO']=='PSE', 'Country']='State of Palestine'
    logging.info('Country changed from West Bank and Gaza to State of Palestine')

    #add Note regarding State of Palestine
    df_tc360.loc[df_tc360['ISO']=='PSE', 'Note']='Only for West Bank and Gaza'
    logging.info('West bank and Gaza note is added')

    df_tc360['Source']='TC360'
    df_tc360.to_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/TC360_dataset.xlsx',index=False)
    logging.info('TC360_data.xlsx saved')

except Exception as e:
    logging.warning(f' !!! WARNING !!! {e}')
    raise e

logging.info('///////////////////////////////////////////////////////\n')
#######################################################################


#### WorldBank WDI

In [6]:
try:
    logging.info('Getting WDI data')
    #get countries codes to add to the url
    df_countries=pd.read_csv('C:/Users/511232/Desktop/Worldwide Governance Indicators/WDI metadata/WDICountry.csv')
    df_countries=df_countries[df_countries['Filter']==1].copy()
    countries=dict(zip(df_countries['Country Code'],df_countries['Short Name']))

    #get indicator codes to add to the url
    df_ind=pd.read_csv('C:/Users/511232/Desktop/Worldwide Governance Indicators/WDI metadata/WDISeries.csv')
    df_ind=df_ind[df_ind['Filter']==1]
    ind=dict(zip(df_ind['Series Code'],df_ind['Indicator Name']))

    ########################################
    #concatenate the country and series codes
    country_codes=''
    i=0
    for k in countries.keys():
        if i==0:
            country_codes=country_codes+k
            i=1
        else:
            country_codes=country_codes+';'+k

    #############################################
    main_url='http://api.worldbank.org/v2/country/'+country_codes+'/indicator/'
    # df_WDI=pd.DataFrame()
    df_list=[]

    for series in ind.keys():
        logging.info(f'getting data for series {ind[series]}')
        page=1
        #loop through pages for each series code
        while page:
            url=main_url+series+f'?format=json&page={page}'

            #get the url for download
            response=requests.get(url)

            #check if valid data was returned
            if len(json.loads(response.content)[1])>0:
                #convert to dataframe and append to df_list
                df=pd.DataFrame.from_dict(json.loads(response.content)[1], orient='columns')
                df_list.append(df)
                page+=1
                total_pages=page
            else:
                page=False
                logging.info(f'{ind[series]} downloaded with total {total_pages} pages')
                logging.info('--------------------------------------')
    
    #concatenate the dataframes
    df_WDI=pd.concat(df_list)
    #extract Indicator, Type and Country
    df_WDI['Indicator']=df_WDI['indicator'].map(lambda x: x['value'].split(':')[0])

    def extract(x):
        if len(x['value'].split(':'))>1:
            return(x['value'].split(':')[1])
        else:
            return('')

    df_WDI['Type']=df_WDI['indicator'].map(extract)
    df_WDI['Country']=df_WDI['country'].map(lambda x: x['value'])
    df_WDI.drop(['country', 'indicator'], axis=1, inplace=True)
    logging.info('Indicator, Type and Country successfully extracted')

    #rename columns
    df_WDI.rename({'countryiso3code':'ISO', 'date':'Year','value':'Value'},axis=1, inplace=True)
    
    #change PSE to State of Palestine
    df_WDI.loc[df_WDI['ISO']=='PSE', 'Country']='State of Palestine'
    logging.info('Country changed from West Bank and Gaza to State of Palestine')

    #add Note regarding State of Palestine
    df_WDI.loc[df_WDI['ISO']=='PSE', 'Note']='Only for West Bank and Gaza'
    logging.info('West bank and Gaza note is added')

    #add source
    df_WDI['Source']='WDI'
    df_WDI.to_excel('C:/Users/511232/Desktop/Worldwide Governance Indicators/CODES/Datasets/WDI.xlsx', index=False)
    logging.info('WDI.xlsx saved')

except Exception as e:
    logging.warning(f'!!! WARNING !!!! {e}')
    raise e
    
